8. Haga un modelo usando validación cruzada, compare los resultados de este con los del
modelo anterior. ¿Cuál funcionó mejor?

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Cargar los datos limpios
df_cleaned = pd.read_csv("../dataset/train_cleaned.csv")

# Separar características (X) y variable objetivo (y)
X_cleaned = df_cleaned.drop(columns=['SalePrice'])
y_cleaned = df_cleaned['SalePrice']

# Identificar columnas categóricas
categorical_columns = X_cleaned.select_dtypes(include=['object']).columns.tolist()

# Convertir variables categóricas en variables numéricas con one-hot encoding
X_cleaned_encoded = pd.get_dummies(X_cleaned, columns=categorical_columns)

# División en conjunto de entrenamiento y prueba (80%-20%)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned_encoded, y_cleaned, test_size=0.2, random_state=42)

# Definir y entrenar el modelo KNN
knn_cleaned = KNeighborsRegressor(n_neighbors=5, metric='minkowski')
knn_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Predicción en el conjunto de prueba
y_pred_cleaned = knn_cleaned.predict(X_test_cleaned)

# Calcular RMSE sin validación cruzada
rmse_knn_cleaned = np.sqrt(mean_squared_error(y_test_cleaned, y_pred_cleaned))
print(f"RMSE del modelo KNN sin validación cruzada: {rmse_knn_cleaned:.2f}")

# Aplicar validación cruzada con 10 folds
kf_cleaned = KFold(n_splits=10, shuffle=True, random_state=42)
cv_scores_cleaned = cross_val_score(knn_cleaned, X_cleaned_encoded, y_cleaned, scoring='neg_root_mean_squared_error', cv=kf_cleaned)

# Calcular RMSE promedio con validación cruzada
rmse_mean_cleaned = -cv_scores_cleaned.mean()
rmse_std_cleaned = cv_scores_cleaned.std()

print(f"RMSE con validación cruzada (10 folds): {rmse_mean_cleaned:.2f}")
print(f"Desviación estándar del RMSE con validación cruzada: {rmse_std_cleaned:.2f}")

# Comparación de modelos
print("\nComparación de RMSE:")
print(f"Modelo sin validación cruzada: {rmse_knn_cleaned:.2f}")
print(f"Modelo con validación cruzada: {rmse_mean_cleaned:.2f}")


RMSE del modelo KNN sin validación cruzada: 48941.00
RMSE con validación cruzada (10 folds): 45263.53
Desviación estándar del RMSE con validación cruzada: 6684.58

Comparación de RMSE:
Modelo sin validación cruzada: 48941.00
Modelo con validación cruzada: 45263.53


El modelo KNN con validación cruzada de 10 folds obtuvo un **RMSE de 45,263.53**, mientras que el modelo sin validación cruzada tuvo un **RMSE de 48,941.00**, indicando que la validación cruzada mejoró la precisión del modelo al reducir el error en aproximadamente **3,677.47 unidades**. Además, la **desviación estándar del RMSE (6,684.58)** sugiere que el rendimiento del modelo es relativamente estable en los distintos conjuntos de validación. Estos resultados demuestran que la validación cruzada ayuda a mejorar la generalización del modelo, evitando el sobreajuste y proporcionando predicciones más confiables en datos nuevos. Para futuras mejoras, se podría optimizar los hiperparámetros de KNN o probar modelos más complejos como **Random Forest** o **Gradient Boosting**.

9. Tanto para los modelos de regresión como de clasificación, pruebe con varios valores de los
hiperparámetros ¿Qué parámetros pueden tunearse en un KNN?, use el mejor modelo del
tuneo, ¿Mejoraron los resultados usando el mejor modelo ahora? Explique

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Cargar los datos limpios
df_cleaned = pd.read_csv("../dataset/train_cleaned.csv")

# Separar características (X) y variable objetivo (y)
X_cleaned = df_cleaned.drop(columns=['SalePrice'])
y_cleaned = df_cleaned['SalePrice']

# Identificar columnas categóricas
categorical_columns = X_cleaned.select_dtypes(include=['object']).columns.tolist()

# Convertir variables categóricas en variables numéricas con one-hot encoding
X_cleaned_encoded = pd.get_dummies(X_cleaned, columns=categorical_columns)

# División en conjunto de entrenamiento y prueba (80%-20%)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned_encoded, y_cleaned, test_size=0.2, random_state=42)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Definir el modelo KNN
knn_tuned = KNeighborsRegressor()

# Realizar búsqueda de hiperparámetros con validación cruzada (10 folds)
grid_search = GridSearchCV(knn_tuned, param_grid, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1)
grid_search.fit(X_train_cleaned, y_train_cleaned)

# Obtener los mejores hiperparámetros y el mejor resultado
best_params = grid_search.best_params_
best_rmse = -grid_search.best_score_

print(f"Mejores hiperparámetros: {best_params}")
print(f"Mejor RMSE con validación cruzada: {best_rmse:.2f}")


Mejores hiperparámetros: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
Mejor RMSE con validación cruzada: 41572.45


Después de ajustar los hiperparámetros del modelo KNN, se logró una mejora significativa en el rendimiento. El mejor modelo encontrado utiliza la métrica de distancia **Manhattan**, un número óptimo de **9 vecinos (`k=9`)**, y un esquema de ponderación basado en la distancia en lugar de pesos uniformes. Estos cambios redujeron el **RMSE de 45,263.53 a 41,572.45**, lo que representa una mejora del **8.1%** en la precisión del modelo. El uso de la distancia Manhattan permitió capturar mejor las relaciones en los datos, mientras que aumentar el número de vecinos suavizó las predicciones y redujo la sensibilidad al ruido. Además, ponderar los vecinos por distancia otorgó mayor peso a los más cercanos, generando estimaciones más precisas. Estos resultados demuestran que la **optimización de hiperparámetros es clave para mejorar la generalización del modelo**, haciendo que las predicciones sean más confiables y precisas.